In [ ]:
# Load a pre-trained ResNet18 model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import torchvision
import os
import shutil
from sklearn.model_selection import train_test_split
import zipfile
from models import CustomResNet18

In [6]:

with zipfile.ZipFile('../archive.zip','r') as my_zip:
  my_zip.extractall('data')


In [7]:
# Path to the original dataset
data_dir = 'data/Shoe vs Sandal vs Boot Dataset'

# Paths where the train and test datasets will be saved
train_dir = 'data/train'
test_dir = 'data/test'

# Make sure the train/test directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List of class folders (shoe, sandal, boot)
class_names = ['Shoe', 'Sandal', 'Boot']

# Split ratio
train_ratio = 0.8  # 80% training data, 20% test data

# Function to split and move files
def split_data(class_name):
    # Get the list of files in the class directory
    class_path = os.path.join(data_dir, class_name)

    files = os.listdir(class_path)

    # Split the data into train and test sets
    train_files, test_files = train_test_split(files, train_size=train_ratio, random_state=42)

    # Create class-specific directories in train and test folders
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Move training files
    for file in train_files:
        src_path = os.path.join(class_path, file)
        dst_path = os.path.join(train_class_dir, file)
        shutil.move(src_path, dst_path)

    # Move testing files
    for file in test_files:
        src_path = os.path.join(class_path, file)
        dst_path = os.path.join(test_class_dir, file)
        shutil.move(src_path, dst_path)

# Loop over each class and split its data
for class_name in class_names:
    split_data(class_name)

print("Dataset successfully split into training and testing sets!")


Dataset successfully split into training and testing sets!


In [8]:
BATCH_SIZE=256

data_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((100,100)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]
        ),
    ]
)

train_dir = 'data/train'
test_dir = 'data/test'

train_dataset = datasets.ImageFolder(train_dir, transform=data_transform)
test_dataset = datasets.ImageFolder(test_dir,transform=data_transform)




train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset_loader  = torch.utils.data.DataLoader(test_dataset , batch_size=BATCH_SIZE, shuffle=False)
dataloaders = {'train': train_dataset_loader,'test': test_dataset_loader}

In [9]:
train_dataset_loader_batch = next(iter(train_dataset_loader))

In [10]:
train_dataset_loader_batch

[tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
 
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
 
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]],
 
 
         [[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1

In [16]:
model = CustomResNet18(3)

In [21]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model.resnet.fc.parameters(), 'lr': 1e-3},
    {'params': model.resnet.layer3.parameters(), 'lr': 1e-4},
    {'params': model.resnet.layer4.parameters(), 'lr': 1e-4}
])


In [ ]:
# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0

        for images, labels in train_loader:
            # Move data to GPU if available
            images, labels = images.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Set device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
train_model(model, train_dataset_loader, criterion, optimizer, num_epochs=5)


Epoch [1/5], Loss: 0.4576
Epoch [2/5], Loss: 0.2623
Epoch [3/5], Loss: 0.2387


In [1]:
# Test function
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to calculate gradients during testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on test images: {100 * correct / total}%')

# Test the model
test_model(model, test_dataset_loader)


NameError: name 'model' is not defined

In [ ]:
torch.save(model.state_dict(), 'best_model.pth')


